# Counterfactual Modeling with a Biochemical Network

This workflow demonstrates counterfactual modeling using a [dynamic model of biochemical signaling in lung cancer](https://wwwdev.ebi.ac.uk/biomodels/BIOMD0000000427).

## Model background

### What is this?


This is a dynamic model of cell signaling in lung cancer by [Bianconi et al 2012](https://wwwdev.ebi.ac.uk/biomodels/BIOMD0000000427). Visualization of the model: ![bianconi_viz](https://i.imgur.com/6KUXKsy.png)

The nodes 'sos', 'ras', 'pi3k', 'akt', 'raf', 'mek', 'erk', and 'p90' represent concentrations of enzymatically active proteins at steady state.  This dynamic model is specified as a set of species and reactions between species using Michaelis-Menton kinetics.  The model can be downloaded from [biomodels.org](https://wwwdev.ebi.ac.uk/biomodels/BIOMD0000000427) as a file written in a markup language called SBML, which can be compiled with various software tools.

A module in this repository called `cancer_signaling` uses structural causal modeling to represent the system at *steady state*.  This approach assumes the system is stochastic, and models the probability distribution of the concentrations of the proteins.  The derivations underlying the model can be found in the *bianconi_math* document.

### Counterfactual reasoning on a biochemical model can inform experimental design.

Suppose an experimentalist wanted to do an experiment on the system (eg. forcing a variable's value to increase or knocking it out).  Furthermore, suppose she already has data collected under entirely different conditions than in the proposed experiment.

Counterfactual reasoning could simulate from a probability distribution representing the outcome of the experiment prior to spending the resources on the experiment.  This could save resources by, for example, prioritizing experiments more likely to produce interesting discoveries.

### Who would use this?

This type of dynamic model simulates biochemical reactions in cells. They are used in fields such as drug discovery and synthetic biology to model the effects of an intervention in the cellular system (such as a candidate compound or manipulation of the genetic machinery).

In [16]:
from math import exp

from pprint import pprint
import pyro
from pyro.distributions import LogNormal
from pyro import condition, do, infer, sample
from pyro.infer import EmpiricalMarginal
from pyro.infer.mcmc import MCMC
from pyro.infer.mcmc.nuts import NUTS
import torch

from causal_demon.transmitters import cancer_signaling

from matplotlib import pyplot as plt
%matplotlib inline


### Helper functions for inference

Since the inference algorithm is not the focus of this tutorial, I hide the choice of algorithm in an abstraction.

In [27]:
def infer_noise(model):
    nuts_kernel = NUTS(model, adapt_step_size=True)
    return MCMC(nuts_kernel, num_samples=500, warmup_steps=300)

def infer_noise2(model):
    return pyro.infer.Importance(model, num_samples=1000)

In [3]:
def hist(marginal, name):
    plt.hist([marginal() for _ in range(5000)])
    plt.title("Marginal Histogram of {}".format(name))
    plt.xlabel("concentration")
    plt.ylabel("#")

# Simulating "wild type" data from the model

"Wild type" refers to data forward-generated from the model. 

Pass in noise variables that capture the variation in the model.  Each noise term is modeled with a weakly informed prior. The `independent` method reshapes the distribution such that data replicates will be independent.

In [4]:
noise_vars = ['N_egf', 'N_igf', 'N_sos', 'N_ras', 'N_pi3k', 'N_akt', 'N_raf', 'N_mek', 'N_erk']
noise_priors = {N: LogNormal(0, 10).independent() for N in noise_vars}

`sample_shape` captures the dimensions of the data. Here we simulate 4 observations.

In [5]:
cancer_signaling(noise_priors, sample_shape=torch.Size([4]))

{'egf': tensor([ 1.3451e-04,  8.5265e-01, -5.9698e-07,  1.2893e+05]),
 'igf': tensor([1.1811e+09, 8.2959e-05, 2.6991e-05, 4.0145e+07]),
 'sos': tensor([1.1957e+05, 2.6486e-01, 3.8057e-03, 1.1350e+05]),
 'ras': tensor([55184.7930,     1.4579, 77630.7500, 53698.1367]),
 'pi3k': tensor([ 120010.9844,    1179.5409, 4540044.5000,  119999.7891]),
 'akt': tensor([405095.7500,  29134.3945, 598950.0625, 437960.0312]),
 'raf': tensor([2171.0269,    0.1378, 1444.7947, 1316.2133]),
 'mek': tensor([77864.0000,     4.9157, 47464.5547, 43546.7773]),
 'erk': tensor([428376.6875,     94.5480, 362289.4375, 349577.0000])}

# Simulating an intervention experiment with the do-operator

Suppose we have an *intervention query*:

### Intervention Query: *What would happen if we blocked IGF and varied the amount of EGF?*

We could answer this query with an experiment:
* 3 conditions: EGF is varied from low (set to 800), medium (8000), to high (80000) concentrations
* 2 replicates of each condition.
* IGF is blocked (set to 0).

Prior to running the experiment, we can simulate the experiment using this model.  

**Why simulate experiments?**  Interventions with the `do` operator allow you to simulate the results of an experiment prior to spending the resources to conduct the experiment.

The interventions on IFG and EGF in this example are one set from a very large set of possible interventions.  Which sent of interventions is the best?  You can answer this by defining a cost function on the experimental results, applying that function to simulated data, and iteratively searching for a set of interventions that optimizes that cost function. 

In [6]:
conditions = {
    'egf': torch.tensor(
        [
            [800., 8000., 80000.],
            [800., 8000., 80000.],
        ]
    ),
    'igf': torch.zeros([2, 3])
}
initial_experiment = do(cancer_signaling, data=conditions)
initial_results = initial_experiment(noise_priors, torch.Size([2, 3]))
pprint(initial_results)

{'akt': tensor([[391338.7188, 404718.7812, 407006.2500],
        [392281.4375, 403664.8438, 573941.7500]]),
 'egf': tensor([[  800.,  8000., 80000.],
        [  800.,  8000., 80000.]]),
 'erk': tensor([[5.5875e+03, 5.6264e+05, 2.0091e+04],
        [9.7106e+05, 9.5360e+08, 5.2224e+05]]),
 'igf': tensor([[0., 0., 0.],
        [0., 0., 0.]]),
 'mek': tensor([[   293.2357, 469678.3750,    956.1735],
        [    52.5269,   1588.2506, 209497.5625]]),
 'pi3k': tensor([[ 108291.1875,  119667.8672,  119870.2812],
        [ 109045.3203,  118715.5391, 1271766.0000]]),
 'raf': tensor([[    8.2239, 60607.8164,    11.9922],
        [    1.4725,    44.3995,  9023.0674]]),
 'ras': tensor([[2.2908e+02, 4.7263e+06, 1.1677e+02],
        [5.5069e+01, 1.1748e+03, 4.3400e+01]]),
 'sos': tensor([[  0.5087,   5.0809, 136.9033],
        [ 51.3112,   6.9473,  50.8112]])}


# Simulating a follow-up experiment using counterfactual inference

Suppose that after that initial experiment, we want to try a new experiment under a different set of conditions.  We could repeat the simulation we just ran under the new conditions, but we can improve the quality of the simulation by using the old data.

However, the old data was collected under different experimental conditions than in the proposed experiment.  This is addressed by posing a counterfactual query to the model:  

### Counterfactual Query: *Given the observations of Erk after fixing IGF to 0, what would Erk have been if IGF were set to 8000?*

The following illustrates how to use the counterfactual simulation algorithm to answer this query.

### Conditioning on previous observations

Firstly, we pretend the simulated data from the previous section is real, and that we only collected data for Erk.

In [14]:
erk_data = initial_results['erk']
erk_data.shape[0]

2

## Step 1: Condition the model of the initial experiment on the observed data

In [18]:
with pyro.iarange("erk", erk_data.shape[0]) as ind:
    evidence = {'erk': initial_results['erk'][ind, :]}
    initial_experiment_conditioned = condition(initial_experiment, data=evidence)

## Step 2: Update the noise priors given the observed data.

In [30]:
model_posterior = infer_noise2(initial_experiment_conditioned).run(noise_priors, sample_shape=torch.Size([2, 3]))
#noise_posteriors = {
#    n: EmpiricalMarginal(model_posterior, sites=n)
#    for n in noise_vars
#}

TypeError: log_prob() got an unexpected keyword argument 'sample_shape'

## Step 3. Apply do-operator to original model to obtain a model of the new experiment

In [ ]:
conditions = {
    'egf': torch.tensor(
        [
            [800., 8000., 80000.],
            [800., 8000., 80000.],
        ]
    ),
    'igf': torch.tensor(
        [
            [8000., 8000., 8000.],
            [8000., 8000., 8000.],
        ]
    ),
}
new_experiment = do(cancer_signaling, data=conditions)

### 4. Pass the updated noise marginals to the model of the new experiment.

In [ ]:
noise_marginals['N_egf'].sample()

In [ ]:
cancer_do_dist = infer_dist(cancer_do, noise_marginals)
erk_cf_marginal = EmpiricalMarginal(cancer_do_dist, sites = 'erk')
hist(erk_cf_marginal, 'Erk')

## Extentions

* Condition on IID data
* Use SVI
* Extend the model to accomadate uncertainty in parameters